### Test dataframe and files for learning more about GoogleMaps and Geomapping

In [1]:
#To setup GMAPS, you either use CONDA or PIP. Exit all Jupyter Notebooks and close. Using GitBash or CommandLine
#CONDA
#$ conda install -c conda-forge gmaps
#
#PIP
#jupyter nbextension enable --py --sys-prefix widgetsnbextension
#pip install gmaps
#jupyter nbextension enable --py --sys-prefix gmaps


In [2]:
#Import the normal dependencies for geomapping in Google
import pandas as pd
import requests
import numpy as np
import json
from pprint import pprint
import os

#Import gmaps and api_key from config.py file. Imports the gmaps function and gkey. NOTE: Can't call api_key, only gkey!
import gmaps
from config import gkey

# Access maps with unique API key. Necessary to disply gmaps properly.
gmaps.configure(api_key = gkey)

In [3]:
#Use Pandas to open the dataframe for all students in grade 5 in the Resources folder. Use the OS module.
#Also try to use the input information from the user!
#filename = input('What CSV file do you want to open? Please give the full name:') This worked!
#Inconvenient to keep entering the filename, so I hardcoded for simplicity!
path = os.path.join('Resources', 'AllStudentsGrade5.csv')

file_df = pd.read_csv(path, header=0)
file_df.head(3)

,CAMPUS,DNAME,CNAME,GRADE,District Type,School Site Street Address,School Site City,School Site Zip,Below Grade Level,Approach Grade Level,Meet Grade Level,Master Grade Level
0,1902103,CAYUGA ISD,CAYUGA ELEM.,5,INDEPENDENT,17750 N US HWY 287,TENNESSEE COLONY,75861,4.0,44.0,41.0,29.0
1,1903102,ELKHART ISD,ELKHART INTERME,5,INDEPENDENT,301 E PARKER ST,ELKHART,75839-9701,3.0,80.0,65.0,43.0
2,1904102,FRANKSTON ISD,FRANKSTON ELEM.,5,INDEPENDENT,100 PERRY ST,FRANKSTON,75763-0428,7.0,53.0,40.0,26.0


In [4]:
#Worked! Make a short 10 member dataframe for GoogleMap testing.
short_df = file_df.head(10)

In [5]:
short_df.count()

CAMPUS                        10
DNAME                         10
CNAME                         10
GRADE                         10
District Type                 10
School Site Street Address    10
School Site City              10
School Site Zip               10
Below Grade Level             10
Approach Grade Level          10
Meet Grade Level              10
Master Grade Level            10
dtype: int64

In [6]:
#Extract address information for these 10 schools.
address_df = short_df[['CNAME', 'School Site Street Address', 'School Site City', 'School Site Zip']]
address_df.head()

,CNAME,School Site Street Address,School Site City,School Site Zip
0,CAYUGA ELEM.,17750 N US HWY 287,TENNESSEE COLONY,75861
1,ELKHART INTERME,301 E PARKER ST,ELKHART,75839-9701
2,FRANKSTON ELEM.,100 PERRY ST,FRANKSTON,75763-0428
3,NECHES ELEM.,3055 FM 2574,PALESTINE,75803
4,STORY INTERMEDI,5300 S LOOP 256,PALESTINE,75801-2415


In [7]:
#Add latitude and longitude columns for Google Maps functions.
address_df['lat'] = ''
address_df['lng'] = ''
address_df.head()

C:\Users\StevenJBark\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\StevenJBark\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,CNAME,School Site Street Address,School Site City,School Site Zip,lat,lng
0,CAYUGA ELEM.,17750 N US HWY 287,TENNESSEE COLONY,75861,,
1,ELKHART INTERME,301 E PARKER ST,ELKHART,75839-9701,,
2,FRANKSTON ELEM.,100 PERRY ST,FRANKSTON,75763-0428,,
3,NECHES ELEM.,3055 FM 2574,PALESTINE,75803,,
4,STORY INTERMEDI,5300 S LOOP 256,PALESTINE,75801-2415,,


In [8]:
#Generate the target addresses for analysis in Google Geocode API and append to an Address List for checking.
#NOTE: In creating the target_address, I had to cast all of the information as strings for row[1], row[2], and row[3] indexes.
AddressList = []
for index, row in address_df.iterrows():
    target_address = str(row[1]) + ' ' + str(row[2]) + ' TX ' + str(row[3])
    AddressList.append(target_address)
    
    #Generate target urls for submission to Google Geocode
    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?address={target_address}&key={gkey}')
    #print(target_url) for printing the target_url and checking formatting is OK.
    

    #Setup search for each address based on target_url made in each iterrows interation. Ouptput in json format.
    schoolcoords = requests.get(target_url).json()
    
    #Extract the latitude and longitude information and try, except to add these coordinates to the address_df dataframe.
    try:
        address_df.loc[index, 'lat'] = schoolcoords['results'][0]['geometry']['location']['lat'] #latitude
        address_df.loc[index, 'lng'] = schoolcoords['results'][0]['geometry']['location']['lng'] #longitude
        
    except:KeyError
        
    #If we need to generate a tuple for Google Map functions directly, this might work:
    #address_df.loc[index, 'columnheader'] = (school_1['results'][0]['geometry']['location']['lat'], school_1['results'][0]['geometry']['location']['lng'])

C:\Users\StevenJBark\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\StevenJBark\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\StevenJBark\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
AddressList

['17750 N US HWY 287 TENNESSEE COLONY TX 75861',
 '301 E PARKER ST ELKHART TX 75839-9701',
 '100 PERRY ST FRANKSTON TX 75763-0428',
 '3055 FM 2574 PALESTINE TX 75803',
 '5300 S LOOP 256 PALESTINE TX 75801-2415',
 '2305 SALT WORKS RD PALESTINE TX 75803-3955',
 '5765 E STATE HWY 294 ELKHART TX 75839-9802',
 '316 SW 5TH ST ANDREWS TX 79714-6905',
 '602 S RAGUET ST LUFKIN TX 75904',
 '536 FM 3258 LUFKIN TX 75904']

In [10]:
address_df

,CNAME,School Site Street Address,School Site City,School Site Zip,lat,lng
0,CAYUGA ELEM.,17750 N US HWY 287,TENNESSEE COLONY,75861,31.923,-95.9239
1,ELKHART INTERME,301 E PARKER ST,ELKHART,75839-9701,31.6281,-95.579
2,FRANKSTON ELEM.,100 PERRY ST,FRANKSTON,75763-0428,32.0621,-95.5043
3,NECHES ELEM.,3055 FM 2574,PALESTINE,75803,31.8706,-95.4921
4,STORY INTERMEDI,5300 S LOOP 256,PALESTINE,75801-2415,31.7768,-95.6108
5,WESTWOOD ELEM.,2305 SALT WORKS RD,PALESTINE,75803-3955,31.7488,-95.6796
6,SLOCUM PRE-K -,5765 E STATE HWY 294,ELKHART,75839-9802,31.6321,-95.4621
7,UNDERWOOD ELEM.,316 SW 5TH ST,ANDREWS,79714-6905,32.3142,-102.552
8,SARAH STRINDEN,602 S RAGUET ST,LUFKIN,75904,31.3325,-94.7322
9,W.H. BONNER ELE,536 FM 3258,LUFKIN,75904,31.3142,-94.8452


In [11]:
#Create a tuple listing for each row. Make new column 'lat_lng', then insert tuple for lat,lng to this column.
address_df['lat_lng'] = ''
address_df

C:\Users\StevenJBark\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,CNAME,School Site Street Address,School Site City,School Site Zip,lat,lng,lat_lng
0,CAYUGA ELEM.,17750 N US HWY 287,TENNESSEE COLONY,75861,31.923,-95.9239,
1,ELKHART INTERME,301 E PARKER ST,ELKHART,75839-9701,31.6281,-95.579,
2,FRANKSTON ELEM.,100 PERRY ST,FRANKSTON,75763-0428,32.0621,-95.5043,
3,NECHES ELEM.,3055 FM 2574,PALESTINE,75803,31.8706,-95.4921,
4,STORY INTERMEDI,5300 S LOOP 256,PALESTINE,75801-2415,31.7768,-95.6108,
5,WESTWOOD ELEM.,2305 SALT WORKS RD,PALESTINE,75803-3955,31.7488,-95.6796,
6,SLOCUM PRE-K -,5765 E STATE HWY 294,ELKHART,75839-9802,31.6321,-95.4621,
7,UNDERWOOD ELEM.,316 SW 5TH ST,ANDREWS,79714-6905,32.3142,-102.552,
8,SARAH STRINDEN,602 S RAGUET ST,LUFKIN,75904,31.3325,-94.7322,
9,W.H. BONNER ELE,536 FM 3258,LUFKIN,75904,31.3142,-94.8452,


In [12]:
address_df['lat_lng'] = list(zip(address_df['lat'], address_df['lng']))

C:\Users\StevenJBark\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
address_df

,CNAME,School Site Street Address,School Site City,School Site Zip,lat,lng,lat_lng
0,CAYUGA ELEM.,17750 N US HWY 287,TENNESSEE COLONY,75861,31.923,-95.9239,"(31.9229638, -95.92387049999999)"
1,ELKHART INTERME,301 E PARKER ST,ELKHART,75839-9701,31.6281,-95.579,"(31.6281018, -95.5789828)"
2,FRANKSTON ELEM.,100 PERRY ST,FRANKSTON,75763-0428,32.0621,-95.5043,"(32.0621485, -95.5042737)"
3,NECHES ELEM.,3055 FM 2574,PALESTINE,75803,31.8706,-95.4921,"(31.8706044, -95.49208039999999)"
4,STORY INTERMEDI,5300 S LOOP 256,PALESTINE,75801-2415,31.7768,-95.6108,"(31.7767796, -95.6107632)"
5,WESTWOOD ELEM.,2305 SALT WORKS RD,PALESTINE,75803-3955,31.7488,-95.6796,"(31.748752, -95.679588)"
6,SLOCUM PRE-K -,5765 E STATE HWY 294,ELKHART,75839-9802,31.6321,-95.4621,"(31.6321077, -95.46213639999999)"
7,UNDERWOOD ELEM.,316 SW 5TH ST,ANDREWS,79714-6905,32.3142,-102.552,"(32.31424, -102.5522068)"
8,SARAH STRINDEN,602 S RAGUET ST,LUFKIN,75904,31.3325,-94.7322,"(31.3324899, -94.7321992)"
9,W.H. BONNER ELE,536 FM 3258,LUFKIN,75904,31.3142,-94.8452,"(31.3141608, -94.8452212)"


In [14]:
#See if I can create another list of tuples for GMaps. Make new address dataframe.
address2_df = address_df

In [46]:
#I tried .values, but there output generated a dtype=64-type output at the end. I think this doesn't generate
#a proper list. The following method generates a proper list by making a list of tuples. 
coords = []
for index,row in address2_df.iterrows():
    coords.append((row[4],row[5]))

#Tried to use the .loc method, but this didn't work properly. Better to create a list above and assign the list to
#to a new column!
address2_df['coords'] = coords

C:\Users\StevenJBark\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
address2_df

,CNAME,School Site Street Address,School Site City,School Site Zip,lat,lng,lat_lng,coords
0,CAYUGA ELEM.,17750 N US HWY 287,TENNESSEE COLONY,75861,31.923,-95.9239,"(31.9229638, -95.92387049999999)","(31.9229638, -95.92387049999999)"
1,ELKHART INTERME,301 E PARKER ST,ELKHART,75839-9701,31.6281,-95.579,"(31.6281018, -95.5789828)","(31.6281018, -95.5789828)"
2,FRANKSTON ELEM.,100 PERRY ST,FRANKSTON,75763-0428,32.0621,-95.5043,"(32.0621485, -95.5042737)","(32.0621485, -95.5042737)"
3,NECHES ELEM.,3055 FM 2574,PALESTINE,75803,31.8706,-95.4921,"(31.8706044, -95.49208039999999)","(31.8706044, -95.49208039999999)"
4,STORY INTERMEDI,5300 S LOOP 256,PALESTINE,75801-2415,31.7768,-95.6108,"(31.7767796, -95.6107632)","(31.7767796, -95.6107632)"
5,WESTWOOD ELEM.,2305 SALT WORKS RD,PALESTINE,75803-3955,31.7488,-95.6796,"(31.748752, -95.679588)","(31.748752, -95.679588)"
6,SLOCUM PRE-K -,5765 E STATE HWY 294,ELKHART,75839-9802,31.6321,-95.4621,"(31.6321077, -95.46213639999999)","(31.6321077, -95.46213639999999)"
7,UNDERWOOD ELEM.,316 SW 5TH ST,ANDREWS,79714-6905,32.3142,-102.552,"(32.31424, -102.5522068)","(32.31424, -102.5522068)"
8,SARAH STRINDEN,602 S RAGUET ST,LUFKIN,75904,31.3325,-94.7322,"(31.3324899, -94.7321992)","(31.3324899, -94.7321992)"
9,W.H. BONNER ELE,536 FM 3258,LUFKIN,75904,31.3142,-94.8452,"(31.3141608, -94.8452212)","(31.3141608, -94.8452212)"


In [17]:
#Create coordinates list from lat_lng column, have to use iteration either through Python or through Pandas.
#This example uses Python iteration rather than Pandas through iterrrows().
coordinates = []
for i in range(len(address_df)):
    coordinates.append(address_df['lat_lng'][i])

In [18]:
coordinates

[(31.9229638, -95.92387049999999),
 (31.6281018, -95.5789828),
 (32.0621485, -95.5042737),
 (31.8706044, -95.49208039999999),
 (31.7767796, -95.6107632),
 (31.748752, -95.679588),
 (31.6321077, -95.46213639999999),
 (32.31424, -102.5522068),
 (31.3324899, -94.7321992),
 (31.3141608, -94.8452212)]

In [19]:
#Create a list for the school names to use in symbol identifications.
schoolslist = []
for i in range(len(address_df)):
    schoolslist.append(address_df['CNAME'][i])
schoolslist

['CAYUGA ELEM.',
 'ELKHART INTERME',
 'FRANKSTON ELEM.',
 'NECHES ELEM.',
 'STORY INTERMEDI',
 'WESTWOOD ELEM.',
 'SLOCUM PRE-K -',
 'UNDERWOOD ELEM.',
 'SARAH STRINDEN',
 'W.H. BONNER ELE']

In [20]:
# store the figure in a variable so we can continue to add layers to the same map
fig = gmaps.figure()

# display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Create a marker layer using our coordinates
#markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
#fig.add_layer(markers)

#Addiing infoboxes and symbols to the map in a new symbol_layer. Infobox content shows school at marker when clicked.
#hover_text shows info at marker when hovering. Note: coordinates and schoolslist must have same length and be coordinated
#with each other. Generate from the same dataframe and create a coordinates list of tuples and a name list associated
#with these coordinates list of tuples.
list_of_infoboxes = schoolslist
symbols = gmaps.symbol_layer(coordinates, fill_color='green', stroke_color='green', info_box_content=schoolslist, hover_text=schoolslist)
fig.add_layer(symbols)

#display the figure with the newly added layer.
fig

Figure(layout=FigureLayout(height='420px'))